![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
# Programming Quantum Computers by O'Reilly Media -  [book info](http://shop.oreilly.com/product/0636920167433.do)  - [all code samples](https://oreilly-qc.github.io)

## Code samples for Chapter 12
These code samples were written by Mariia Mykhailova.

### Example 12-1: Complete factoring with Shor

*This example follows [integer factorization example](https://github.com/microsoft/Quantum/tree/master/samples/algorithms/integer-factorization) from official Q# Samples repository.*

In [1]:
// Example 12-1: Complete factoring with Shor

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Oracles;
open Microsoft.Quantum.Characterization;


operation ApplyOrderFindingOracle(generator : Int, modulus : Int, power : Int, target : Qubit[]) : Unit is Adj + Ctl {
    MultiplyByModularInteger(ExpModI(generator, power, modulus), modulus, LittleEndian(target));
}

operation EstimatePeriod(generator : Int, modulus : Int) : Int {
    mutable result = 1;
    let bitsize = BitSizeI(modulus);
    let bitsPrecision = 2 * bitsize + 1;

    repeat {
        mutable dyadicFractionNum = 0;

        using (eigenstateRegister = Qubit[bitsize]) {
            let eigenstateRegisterLE = LittleEndian(eigenstateRegister);
            ApplyXorInPlace(1, eigenstateRegisterLE);

            let oracle = DiscreteOracle(ApplyOrderFindingOracle(generator, modulus, _, _));

            using (register = Qubit[bitsPrecision]) {
                let dyadicFractionNumerator = LittleEndian(register);

                QuantumPhaseEstimation(oracle, eigenstateRegisterLE!, LittleEndianAsBigEndian(dyadicFractionNumerator));

                set dyadicFractionNum = MeasureInteger(dyadicFractionNumerator);
            }

            ResetAll(eigenstateRegister);
        }

        if (dyadicFractionNum == 0) {
            fail "We measured 0 for the numerator";
        }

        Message($"Estimated eigenvalue is {dyadicFractionNum}/2^{bitsPrecision}.");

        let (numerator, period) = (ContinuedFractionConvergentI(Fraction(dyadicFractionNum, 2 ^ bitsPrecision), modulus))!;
        let (numeratorAbs, periodAbs) = (AbsI(numerator), AbsI(period));

        Message($"Estimated divisor of period is {periodAbs}, " + $" we have projected on eigenstate marked by {numeratorAbs}.");

        set result = (periodAbs * result) / GreatestCommonDivisorI(result, periodAbs);
    }
    until (ExpModI(generator, result, modulus) == 1)
    fixup {
        Message("It looks like the period has divisors and we have found only a divisor of the period. Trying again ...");
    }

    return result;
}


operation ShorsAlgorithm (periodFindingRoutine : ((Int, Int) => Int)) : Unit {
    // This code is a general-purpose implementation of Shor's algorithm, so it's not optimized for coprime=2
    // N = 21 runs a fairly long time, and we don't recommend trying N = 35 and higher.
    let N = 15;
    let coprime = 2;
    
    Message($"Estimating period of {coprime}...");

    // Call Quantum Period finding algorithm for `coprime` mod `N`.
    let period = periodFindingRoutine(coprime, N);

    // Period finding reduces to factoring only if period is even
    if (period % 2 == 0) {

        // Compute `coprimeCandidate` ^ `period/2` mod `N`
        let halfPower = ExpModI(coprime, period / 2, N);

        if (halfPower != N - 1) {
            // When the halfPower is not -1 mod N,
            // halfPower-1 or halfPower+1 share non-trivial divisor with `N`
            let factor = MaxI(GreatestCommonDivisorI(halfPower - 1, N), 
                              GreatestCommonDivisorI(halfPower + 1, N));

            // Return computed non-trivial factors.
            Message($"Factors are {factor} and {N / factor}");
        } else {
            // If we are unlucky, halfPower is just -1 mod N
            // This is a trivial case not useful for factoring
            fail "Residue xᵃ = -1 (mod N) where a is a period.";
        }
    } else {
        // When period is odd we have to pick another number to estimate
        // period of and start over.
        fail "Period is odd.";
    }

}

operation RunShorsAlgorithmQPU () : Unit {
    ShorsAlgorithm(EstimatePeriod);
}

ApplyOrderFindingOracle, EstimatePeriod, RunShorsAlgorithmQPU, ShorsAlgorithm

In [2]:
%simulate RunShorsAlgorithmQPU

Estimating period of 2...
Estimated eigenvalue is 128/2^9.
Estimated divisor of period is 4,  we have projected on eigenstate marked by 1.
Factors are 5 and 3


()

### Example 12-2: Factoring without a QPU

In [3]:
// Example 12-2: Factoring without a QPU

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;

operation EstimatePeriodNoQPU(generator : Int, modulus : Int) : Int {
    let bitsize = BitSizeI(modulus);
    let bitsPrecision = 2 * bitsize + 1;
    mutable work = 1;
    for (iter in 0 .. (2 <<< bitsPrecision) - 1) {
        set work = (work * generator) % modulus;
        if (work == 1) {
            return iter + 1;
        }
    }
    return 0;
}

operation RunShorsAlgorithmNoQPU () : Unit {
    ShorsAlgorithm(EstimatePeriodNoQPU);
}

EstimatePeriodNoQPU, RunShorsAlgorithmNoQPU

In [4]:
%simulate RunShorsAlgorithmNoQPU

Estimating period of 2...
Factors are 5 and 3


()